PyGSLIB
========

Introduction
---------------

This is a simple example on how to use raw pyslib to compute variograms



In [21]:
#general imports
import pygslib                  

Getting the data ready for work
---------
If the data is in GSLIB format you can use the function `pygslib.gslib.read_gslib_file(filename)` to import the data into a Pandas DataFrame. 


In [22]:
#get the data in gslib format into a pandas Dataframe
mydata= pygslib.gslib.read_gslib_file('../datasets/cluster.dat')  

In [23]:
# This is a 2D file, in this GSLIB version we require 3D data and drillhole name or domain code
# so, we are adding constant elevation = 0 and a dummy BHID = 1 
mydata['Zlocation']=0.
mydata['bhid']=1.

# printing to verify results
print ' \n **** 5 first rows in my datafile \n\n  ', mydata.head(n=5)

 
 **** 5 first rows in my datafile 

      Xlocation  Ylocation  Primary  Secondary  Declustering Weight  Zlocation  \
0       39.5       18.5     0.06       0.22                1.619        0.0   
1        5.5        1.5     0.06       0.27                1.619        0.0   
2       38.5        5.5     0.08       0.40                1.416        0.0   
3       20.5        1.5     0.09       0.39                1.821        0.0   
4       27.5       14.5     0.09       0.24                1.349        0.0   

   bhid  
0   1.0  
1   1.0  
2   1.0  
3   1.0  
4   1.0  


## Testing variogram function gamv

In [54]:
# these are the parameters we need. Note that at difference of GSLIB this dictionary also stores 
# the actual data (ex, X, Y, etc.). 

#important! python is case sensitive 'bhid' is not equal to 'BHID'


parameters = {
            'x'    : mydata['Xlocation'].values,
            'y'    : mydata['Ylocation'].values,
            'z'    : mydata['Zlocation'].values, 
            'bhid' : mydata['bhid'].values,
            'vr'   : mydata['Primary'].values,
            'tmin' : -1.0e21,
            'tmax' :  1.0e21,
            'nlag' :  10,
            'xlag' :  1,
            'ndir' : 10,
            'ndip' :  10,
            'orgdir': 0.,
            'orgdip': 0.,
            'isill' : 1,
            'sills' : [mydata['Primary'].var()],
            'ivtail' : [1],
            'ivhead' : [1],
            'ivtype' : [1]
        }


In [55]:
#Now we are ready to calculate the veriogram
np, dis, gam, hm, tm, hv, tv = pygslib.gslib.gamv3D(parameters)

In [56]:
np

array([[[[ 140.],
         [  14.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.]],

        [[  19.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.]],

        [[   9.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.]],

        [[  13.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.]],

        [[   7.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],
         [   0.]],

        [[  12.],
         [   0.],
         [   0.],
         [   0.],
         [   0.],


In [62]:
# create structured grid with data 

import vtk
import vtk.util.numpy_support as vtknumpy
import math
import numpy as np

XYZPts = vtk.vtkPoints()
XYZPts.SetNumberOfPoints(parameters['ndip']*parameters['nlag']*(parameters['ndir']+1))

angdir = (math.pi/180.)*360./(parameters['ndir'])
angdip = (math.pi/180.)*90./(parameters['ndip']-1)
orgdir = parameters['orgdir'] * math.pi/180.
orgdip = parameters['orgdip'] * math.pi/180.

id=-1
for k in range(parameters['ndip']):
    for j in range(parameters['nlag']):
        for i in range(parameters['ndir']):
            id+=1
             
            x= parameters['xlag']*(j+1)*math.cos(angdir*i-orgdir)*math.cos(angdip*k-orgdip)
            y= parameters['xlag']*(j+1)*math.sin(angdir*i-orgdir)*math.cos(angdip*k-orgdip)
            z= parameters['xlag']*(j+1)*                            math.sin(angdip*k-orgdip)
            
            if i == 0:
                x0=x
                y0=y
                z0=z
            
            print id, i,j,k, angdir*i*(180/math.pi), angdip*k*(180/math.pi),x,y,z
            #print math.cos(angdip*k-orgdip)
            XYZPts.SetPoint(id,x,y,z)
        id+=1    
        XYZPts.SetPoint(id,x0,y0,z0)
            

XYZGrid = vtk.vtkStructuredGrid()
XYZGrid.SetDimensions(parameters['ndir']+1,parameters['nlag'],parameters['ndip'])
XYZGrid.SetPoints(XYZPts)


ptid = np.arange(parameters['ndip']*parameters['nlag']*(parameters['ndir']+1))
cscalars = vtknumpy.numpy_to_vtk(ptid)
cscalars.SetName('PointID|') 
XYZGrid.GetPointData().AddArray(cscalars)


#Write file
writer = vtk.vtkXMLStructuredGridWriter()
writer.SetFileName("output.vts")
writer.SetInputData(XYZGrid)
writer.Write()

0 0 0 0 0.0 0.0 1.0 0.0 0.0
1 1 0 0 36.0 0.0 0.809016994375 0.587785252292 0.0
2 2 0 0 72.0 0.0 0.309016994375 0.951056516295 0.0
3 3 0 0 108.0 0.0 -0.309016994375 0.951056516295 0.0
4 4 0 0 144.0 0.0 -0.809016994375 0.587785252292 0.0
5 5 0 0 180.0 0.0 -1.0 1.22464679915e-16 0.0
6 6 0 0 216.0 0.0 -0.809016994375 -0.587785252292 0.0
7 7 0 0 252.0 0.0 -0.309016994375 -0.951056516295 0.0
8 8 0 0 288.0 0.0 0.309016994375 -0.951056516295 0.0
9 9 0 0 324.0 0.0 0.809016994375 -0.587785252292 0.0
11 0 1 0 0.0 0.0 2.0 0.0 0.0
12 1 1 0 36.0 0.0 1.61803398875 1.17557050458 0.0
13 2 1 0 72.0 0.0 0.61803398875 1.90211303259 0.0
14 3 1 0 108.0 0.0 -0.61803398875 1.90211303259 0.0
15 4 1 0 144.0 0.0 -1.61803398875 1.17557050458 0.0
16 5 1 0 180.0 0.0 -2.0 2.44929359829e-16 0.0
17 6 1 0 216.0 0.0 -1.61803398875 -1.17557050458 0.0
18 7 1 0 252.0 0.0 -0.61803398875 -1.90211303259 0.0
19 8 1 0 288.0 0.0 0.61803398875 -1.90211303259 0.0
20 9 1 0 324.0 0.0 1.61803398875 -1.17557050458 0.0
22 0 2 0 0.0 0.0

172 7 5 1 252.0 10.0 -1.82593399124 -5.6196469848 1.041889066
173 8 5 1 288.0 10.0 1.82593399124 -5.6196469848 1.041889066
174 9 5 1 324.0 10.0 4.78035725027 -3.47313284138 1.041889066
176 0 6 1 0.0 10.0 6.89365427109 0.0 1.21553724367
177 1 6 1 36.0 10.0 5.57708345865 4.05198831495 1.21553724367
178 2 6 1 72.0 10.0 2.13025632311 6.5562548156 1.21553724367
179 3 6 1 108.0 10.0 -2.13025632311 6.5562548156 1.21553724367
180 4 6 1 144.0 10.0 -5.57708345865 4.05198831495 1.21553724367
181 5 6 1 180.0 10.0 -6.89365427109 8.44229163751e-16 1.21553724367
182 6 6 1 216.0 10.0 -5.57708345865 -4.05198831495 1.21553724367
183 7 6 1 252.0 10.0 -2.13025632311 -6.5562548156 1.21553724367
184 8 6 1 288.0 10.0 2.13025632311 -6.5562548156 1.21553724367
185 9 6 1 324.0 10.0 5.57708345865 -4.05198831495 1.21553724367
187 0 7 1 0.0 10.0 7.8784620241 0.0 1.38918542134
188 1 7 1 36.0 10.0 6.37380966703 4.63084378851 1.38918542134
189 2 7 1 72.0 10.0 2.43457865498 7.4928626464 1.38918542134
190 3 7 1 108.0 1

448 8 0 4 288.0 40.0 0.23672075137 -0.7285515594 0.642787609687
449 9 0 4 324.0 40.0 0.61974297293 -0.450269626266 0.642787609687
451 0 1 4 0.0 40.0 1.53208888624 0.0 1.28557521937
452 1 1 4 36.0 40.0 1.23948594586 0.900539252532 1.28557521937
453 2 1 4 72.0 40.0 0.473441502741 1.4571031188 1.28557521937
454 3 1 4 108.0 40.0 -0.473441502741 1.4571031188 1.28557521937
455 4 1 4 144.0 40.0 -1.23948594586 0.900539252532 1.28557521937
456 5 1 4 180.0 40.0 -1.53208888624 1.87626775054e-16 1.28557521937
457 6 1 4 216.0 40.0 -1.23948594586 -0.900539252532 1.28557521937
458 7 1 4 252.0 40.0 -0.473441502741 -1.4571031188 1.28557521937
459 8 1 4 288.0 40.0 0.473441502741 -1.4571031188 1.28557521937
460 9 1 4 324.0 40.0 1.23948594586 -0.900539252532 1.28557521937
462 0 2 4 0.0 40.0 2.29813332936 0.0 1.92836282906
463 1 2 4 36.0 40.0 1.85922891879 1.3508088788 1.92836282906
464 2 2 4 72.0 40.0 0.710162254111 2.1856546782 1.92836282906
465 3 2 4 108.0 40.0 -0.710162254111 2.1856546782 1.92836282906

742 5 7 6 180.0 60.0 -4.0 4.89858719659e-16 6.92820323028
743 6 7 6 216.0 60.0 -3.2360679775 -2.35114100917 6.92820323028
744 7 7 6 252.0 60.0 -1.2360679775 -3.80422606518 6.92820323028
745 8 7 6 288.0 60.0 1.2360679775 -3.80422606518 6.92820323028
746 9 7 6 324.0 60.0 3.2360679775 -2.35114100917 6.92820323028
748 0 8 6 0.0 60.0 4.5 0.0 7.79422863406
749 1 8 6 36.0 60.0 3.64057647469 2.64503363532 7.79422863406
750 2 8 6 72.0 60.0 1.39057647469 4.27975432333 7.79422863406
751 3 8 6 108.0 60.0 -1.39057647469 4.27975432333 7.79422863406
752 4 8 6 144.0 60.0 -3.64057647469 2.64503363532 7.79422863406
753 5 8 6 180.0 60.0 -4.5 5.51091059616e-16 7.79422863406
754 6 8 6 216.0 60.0 -3.64057647469 -2.64503363532 7.79422863406
755 7 8 6 252.0 60.0 -1.39057647469 -4.27975432333 7.79422863406
756 8 8 6 288.0 60.0 1.39057647469 -4.27975432333 7.79422863406
757 9 8 6 324.0 60.0 3.64057647469 -2.64503363532 7.79422863406
759 0 9 6 0.0 60.0 5.0 0.0 8.66025403784
760 1 9 6 36.0 60.0 4.04508497187 2.93

1025 2 3 9 72.0 90.0 7.56873346087e-17 2.32941663698e-16 4.0
1026 3 3 9 108.0 90.0 -7.56873346087e-17 2.32941663698e-16 4.0
1027 4 3 9 144.0 90.0 -1.98152014523e-16 1.43965865561e-16 4.0
1028 5 3 9 180.0 90.0 -2.44929359829e-16 2.99951956532e-32 4.0
1029 6 3 9 216.0 90.0 -1.98152014523e-16 -1.43965865561e-16 4.0
1030 7 3 9 252.0 90.0 -7.56873346087e-17 -2.32941663698e-16 4.0
1031 8 3 9 288.0 90.0 7.56873346087e-17 -2.32941663698e-16 4.0
1032 9 3 9 324.0 90.0 1.98152014523e-16 -1.43965865561e-16 4.0
1034 0 4 9 0.0 90.0 3.06161699787e-16 0.0 5.0
1035 1 4 9 36.0 90.0 2.47690018154e-16 1.79957331951e-16 5.0
1036 2 4 9 72.0 90.0 9.46091682609e-17 2.91177079622e-16 5.0
1037 3 4 9 108.0 90.0 -9.46091682609e-17 2.91177079622e-16 5.0
1038 4 4 9 144.0 90.0 -2.47690018154e-16 1.79957331951e-16 5.0
1039 5 4 9 180.0 90.0 -3.06161699787e-16 3.74939945665e-32 5.0
1040 6 4 9 216.0 90.0 -2.47690018154e-16 -1.79957331951e-16 5.0
1041 7 4 9 252.0 90.0 -9.46091682609e-17 -2.91177079622e-16 5.0
1042 8 4 9 

1

In [60]:
print parameters['ndip']*parameters['nlag']*parameters['ndir']

1000
